In [1]:
!pip install "pymilvus[model]==2.5.10" openai==1.82.0 requests==2.32.3 tqdm==4.67.1 torch==2.7.0

In [2]:
file_path = "./mfd.md"
text_lines = []
with open(file_path, "r") as file:
    file_text = file.read()
text_lines += file_text.split("# ")
len(text_lines)

30

In [3]:
# 定义一个 embedding 模型，使用 milvus_model 来生成文本嵌入。 DefaultEmbeddingFunction 模型
from pymilvus import model as milvus_model
embedding_model = milvus_model.DefaultEmbeddingFunction()

/home/leacock/miniconda3/envs/deepseek/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
test_embedding = embedding_model.encode_queries(["This is a test"])[0]
embedding_dim = len(test_embedding)
# 将数据加载到 Milvus
# 建立向量数据库
from pymilvus import MilvusClient
milvus_client = MilvusClient(uri="./milvus_demo.db")
# 创建 Collection
collection_name = "my_rag_collection_mfd"
if milvus_client.has_collection(collection_name):
    milvus_client.drop_collection(collection_name)
milvus_client.create_collection(
    collection_name=collection_name,
    dimension=embedding_dim,
    metric_type="IP",  # 内积距离
    consistency_level="Strong",  # 支持的值为 (`"Strong"`, `"Session"`, `"Bounded"`, `"Eventually"`)。更多详情请参见 https://milvus.io/docs/consistency.md#Consistency-Level。
)
# encode 数据 并插入数据库
from tqdm import tqdm
data = []
doc_embeddings = embedding_model.encode_documents(text_lines)
for i, line in enumerate(tqdm(text_lines, desc="Creating embeddings")):
    data.append({"id": i, "vector": doc_embeddings[i], "text": line})
milvus_client.insert(collection_name=collection_name, data=data)

Creating embeddings: 100%|██████████| 30/30 [00:00<00:00, 168898.15it/s]


{'insert_count': 30, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29], 'cost': 0}

In [5]:
# 构建 RAG
question = "居住权是什么，购买房屋时对于居住权应该注意什么"
# 在 collection 中搜索该问题，并检索语义上最匹配的前5个结果
search_res = milvus_client.search(
    collection_name=collection_name,
    data=embedding_model.encode_queries(
        [question]
    ),  # 将问题转换为嵌入向量
    limit=5,  # 返回前5个结果
    search_params={"metric_type": "IP", "params": {}},  # 内积距离
    output_fields=["text"],  # 返回 text 字段
)
# 查看下，查询的搜索结果
import json
retrieved_lines_with_distances = [
    (res["entity"]["text"], res["distance"]) for res in search_res[0]
]
print(json.dumps(retrieved_lines_with_distances, indent=4, ensure_ascii=False))

[
    [
        "第三章 合同的变更和转让\n\n**第五百四十八条** 当事人协商一致，可以变更合同。\n\n**第五百四十九条** 当事人对合同变更的内容约定不明确的，推定为未变更。\n\n**第五百五十条** 债权人可以将合同的权利全部或者部分转让给第三人，但是有下列情形之一的除外：\n（一）根据合同性质不得转让；\n（二）按照当事人约定不得转让；\n（三）依照法律规定不得转让。\n债权人转让权利的，应当通知债务人。未经通知，该转让对债务人不发生效力。\n\n**第五百五十一条** 债权人转让权利的，受让人取得与债权有关的从权利，但是该从权利专属于债权人自身的除外。\n\n**第五百五十二条** 债务人将合同的义务全部或者部分转让给第三人的，应当经债权人同意。\n\n**第五百五十三条** 债务人转让义务的，新债务人可以主张原债务人对债权人的抗辩。\n新债务人承担债务的，应当承担与主债务有关的从债务，但是该从债务专属于原债务人自身的除外。\n\n**第五百五十四条** 当事人一方经对方同意，可以将自己在合同中的权利和义务一并转让给第三人。\n\n**第五百五十五条** 权利和义务一并转让的，适用债权转让、债务转让的有关规定。\n\n**第五百五十六条** 合同变更的，不影响当事人请求损害赔偿的权利。\n\n###",
        0.5734704732894897
    ],
    [
        "第二节 土地承包经营权\n\n**第三百五十八条** 土地承包经营权人依照法律规定，对集体所有的土地和国家所有由集体经济组织承包的土地，享有占有、使用和收益的权利，有权从事种植业、林业、畜牧业等农业生产。\n\n**第三百五十九条** 土地承包经营权人可以按照法律规定，将土地承包经营权互换、转让。\n当事人通过互换、转让方式取得土地承包经营权的，未经登记，不得对抗善意第三人。\n\n**第三百六十条** 土地承包经营权人依法对承包地享有占有、使用、收益的权利。\n土地承包经营权人可以采取出租、入股或者其他方式流转土地经营权。\n\n**第三百六十一条** 土地承包经营权人可以根据法律规定，将土地承包经营权互换、转让。\n互换、转让后，土地承包经营权人依法享有土地承包经营权。\n\n**第三百六十二条** 土地经营权人有权在合同约定的期

In [6]:
# 使用 LLM 获取 RAG 响应
# 将检索到的文档转换为字符串格式
context = "\n".join(
    [line_with_distance[0] for line_with_distance in retrieved_lines_with_distances]
)
context

'第三章 合同的变更和转让\n\n**第五百四十八条** 当事人协商一致，可以变更合同。\n\n**第五百四十九条** 当事人对合同变更的内容约定不明确的，推定为未变更。\n\n**第五百五十条** 债权人可以将合同的权利全部或者部分转让给第三人，但是有下列情形之一的除外：\n（一）根据合同性质不得转让；\n（二）按照当事人约定不得转让；\n（三）依照法律规定不得转让。\n债权人转让权利的，应当通知债务人。未经通知，该转让对债务人不发生效力。\n\n**第五百五十一条** 债权人转让权利的，受让人取得与债权有关的从权利，但是该从权利专属于债权人自身的除外。\n\n**第五百五十二条** 债务人将合同的义务全部或者部分转让给第三人的，应当经债权人同意。\n\n**第五百五十三条** 债务人转让义务的，新债务人可以主张原债务人对债权人的抗辩。\n新债务人承担债务的，应当承担与主债务有关的从债务，但是该从债务专属于原债务人自身的除外。\n\n**第五百五十四条** 当事人一方经对方同意，可以将自己在合同中的权利和义务一并转让给第三人。\n\n**第五百五十五条** 权利和义务一并转让的，适用债权转让、债务转让的有关规定。\n\n**第五百五十六条** 合同变更的，不影响当事人请求损害赔偿的权利。\n\n###\n第二节 土地承包经营权\n\n**第三百五十八条** 土地承包经营权人依照法律规定，对集体所有的土地和国家所有由集体经济组织承包的土地，享有占有、使用和收益的权利，有权从事种植业、林业、畜牧业等农业生产。\n\n**第三百五十九条** 土地承包经营权人可以按照法律规定，将土地承包经营权互换、转让。\n当事人通过互换、转让方式取得土地承包经营权的，未经登记，不得对抗善意第三人。\n\n**第三百六十条** 土地承包经营权人依法对承包地享有占有、使用、收益的权利。\n土地承包经营权人可以采取出租、入股或者其他方式流转土地经营权。\n\n**第三百六十一条** 土地承包经营权人可以根据法律规定，将土地承包经营权互换、转让。\n互换、转让后，土地承包经营权人依法享有土地承包经营权。\n\n**第三百六十二条** 土地经营权人有权在合同约定的期限内占有农村土地，开展农业生产经营并取得收益。\n\n**第三百六十三条** 土地经营权人可以依照法律规定，将土地经营权转让、出租、

In [7]:
question

'居住权是什么，购买房屋时对于居住权应该注意什么'

In [8]:
SYSTEM_PROMPT = """
Human: 你是一个 AI 助手。你能够从提供的上下文段落片段中找到问题的答案。
"""
USER_PROMPT = f"""
请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。最后追加原始回答的英文翻译，并用 <translated>和</translated> 标签标注。
<context>
{context}
</context>
<question>
{question}
</question>
<translated>
</translated>
"""
SYSTEM_PROMPT

'\nHuman: 你是一个 AI 助手。你能够从提供的上下文段落片段中找到问题的答案。\n'

In [9]:
USER_PROMPT

'\n请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。最后追加原始回答的英文翻译，并用 <translated>和</translated> 标签标注。\n<context>\n第三章 合同的变更和转让\n\n**第五百四十八条** 当事人协商一致，可以变更合同。\n\n**第五百四十九条** 当事人对合同变更的内容约定不明确的，推定为未变更。\n\n**第五百五十条** 债权人可以将合同的权利全部或者部分转让给第三人，但是有下列情形之一的除外：\n（一）根据合同性质不得转让；\n（二）按照当事人约定不得转让；\n（三）依照法律规定不得转让。\n债权人转让权利的，应当通知债务人。未经通知，该转让对债务人不发生效力。\n\n**第五百五十一条** 债权人转让权利的，受让人取得与债权有关的从权利，但是该从权利专属于债权人自身的除外。\n\n**第五百五十二条** 债务人将合同的义务全部或者部分转让给第三人的，应当经债权人同意。\n\n**第五百五十三条** 债务人转让义务的，新债务人可以主张原债务人对债权人的抗辩。\n新债务人承担债务的，应当承担与主债务有关的从债务，但是该从债务专属于原债务人自身的除外。\n\n**第五百五十四条** 当事人一方经对方同意，可以将自己在合同中的权利和义务一并转让给第三人。\n\n**第五百五十五条** 权利和义务一并转让的，适用债权转让、债务转让的有关规定。\n\n**第五百五十六条** 合同变更的，不影响当事人请求损害赔偿的权利。\n\n###\n第二节 土地承包经营权\n\n**第三百五十八条** 土地承包经营权人依照法律规定，对集体所有的土地和国家所有由集体经济组织承包的土地，享有占有、使用和收益的权利，有权从事种植业、林业、畜牧业等农业生产。\n\n**第三百五十九条** 土地承包经营权人可以按照法律规定，将土地承包经营权互换、转让。\n当事人通过互换、转让方式取得土地承包经营权的，未经登记，不得对抗善意第三人。\n\n**第三百六十条** 土地承包经营权人依法对承包地享有占有、使用、收益的权利。\n土地承包经营权人可以采取出租、入股或者其他方式流转土地经营权。\n\n**第三百六十一条** 土地承包经营权人可以根据法律规定，将土地承包经营权互换、转让。\n互换、转让后，土

In [10]:
# 使用 DeepSeek 提供的 deepseek-chat 模型根据提示生成响应
import os
# 从环境变量获取 DeepSeek API Key
api_key = os.getenv("DEEPSEEK_API_KEY")

from openai import OpenAI

deepseek_client = OpenAI(
    api_key=api_key,
    base_url="https://api.deepseek.com/v1",  # DeepSeek API 的基地址
)
response = deepseek_client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT},
    ],
)
print(response.choices[0].message.content)

根据《中华人民共和国民法典》第三百八十二条至第三百八十七条关于居住权的规定：

**居住权**是指居住权人依照合同约定或遗嘱，对他人的住宅享有占有、使用的权利，以满足生活居住的需要。其核心特征包括：
1. **权利来源**：通过合同或遗嘱设立，需依法登记（未登记不生效）；
2. **权利内容**：仅限占有、使用，不可转让或继承；
3. **限制条件**：设立居住权的住宅原则上不得出租（除非另有约定），且不得改变住宅用途。

**购买房屋时需注意以下居住权问题**：
1. **查验登记**：必须核查不动产登记簿，确认房屋是否已设立居住权（未登记的居住权无效）；
2. **权利限制**：若存在居住权，即使取得所有权，也需容忍居住权人继续占用房屋至约定期限或居住权人死亡；
3. **禁止出租风险**：居住权住宅通常不可出租，可能影响投资回报；
4. **注销义务**：居住权消灭后，应督促办理注销登记（如原居住权人未配合，需通过法律途径解决）。

<translated>
According to Articles 382 to 387 of the Civil Code of the People's Republic of China regarding the right of habitation:

**Right of habitation** refers to the right of a habitation-right holder to possess and use another person's residential property based on contractual agreement or will, for the purpose of meeting living needs. Its key features include:
1. **Source of right**: Established via contract or will, requiring legal registration (invalid without registration);
2. **Content of right**: Limited to possession and use, non-transferable and non-inh